In [1]:
import geopandas as gpd
import numpy as np

In [2]:
gdf = gpd.read_file('./geo_data/kochi_building_from_osm.gpkg')

In [3]:
values_list = [
    "Residential",
    "Commercial",
    "Industrial",
    "Retail",
    "Hospital",
    "School",
    "Recreational",
    "Mixed-Use",
    "Parking Garage",
    "Hotel",
]

In [4]:
gdf['building_types '] = np.random.choice(values_list, size=len(gdf))

In [5]:
gdf['number_of_floors'] = np.random.randint(1, 16, gdf.shape[0])
gdf['building_height'] = gdf['number_of_floors'] * 3 

In [6]:
del gdf["osm_way_id"]
del gdf["building"]


In [7]:
gdf.head()

,osm_id,name,type,amenity,other_tags,geometry,building_types,number_of_floors,building_height
0,1601176,None,multipolygon,place_of_worship,"""religion""=>""hindu""","MULTIPOLYGON (((76.22204 9.99856, 76.22206 9.9...",Industrial,6,18
1,2254959,Bolgatty Palace Hotel & Resort,multipolygon,None,None,"MULTIPOLYGON (((76.26737 9.98423, 76.26744 9.9...",Hotel,4,12
2,5400182,Jawaharlal Nehru Stadium,multipolygon,None,None,"MULTIPOLYGON (((76.30018 9.99822, 76.30004 9.9...",Mixed-Use,7,21
3,5662928,Hanuman Temple,multipolygon,place_of_worship,"""layer""=>""1"",""religion""=>""hindu""","MULTIPOLYGON (((76.28283 9.96882, 76.2829 9.96...",Commercial,7,21
4,5662929,None,multipolygon,None,None,"MULTIPOLYGON (((76.28341 9.96914, 76.28341 9.9...",Recreational,12,36


In [8]:
gdf["name"] = gdf["name"].fillna("unknown")
gdf["amenity"] = gdf["amenity"].fillna("unknown")
gdf["other_tags"] = gdf["other_tags"].fillna("unknown")

In [9]:
gdf = gdf.to_crs(epsg=3857)
gdf['area'] = gdf['geometry'].area

In [10]:
gdf.head()

,osm_id,name,type,amenity,other_tags,geometry,building_types,number_of_floors,building_height,area
0,1601176,unknown,multipolygon,place_of_worship,"""religion""=>""hindu""","MULTIPOLYGON (((8484998.847 1118727.417, 84850...",Industrial,6,18,296.946407
1,2254959,Bolgatty Palace Hotel & Resort,multipolygon,unknown,unknown,"MULTIPOLYGON (((8490044.926 1117106.92, 849005...",Hotel,4,12,2366.468805
2,5400182,Jawaharlal Nehru Stadium,multipolygon,unknown,unknown,"MULTIPOLYGON (((8493696.64 1118688.702, 849368...",Mixed-Use,7,21,30978.103025
3,5662928,Hanuman Temple,multipolygon,place_of_worship,"""layer""=>""1"",""religion""=>""hindu""","MULTIPOLYGON (((8491765.358 1115365.883, 84917...",Commercial,7,21,1002.235757
4,5662929,unknown,multipolygon,unknown,unknown,"MULTIPOLYGON (((8491830.658 1115401.339, 84918...",Recreational,12,36,831.154820


In [11]:
gdf.to_file('./geo_data/Kochi_Buildings.gpkg', driver='GPKG', layer='building')

# Push data to POSTGIS

```
from sqlalchemy import create_engine

# create DB variable
db_user = 'your_username'
db_password = 'your_password'
db_host = 'localhost'
db_port = '5432'
db_name = 'your_database_name'

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Save gdf to Postgis
gdf.to_postgis(name='your_table_name', con=engine, if_exists='replace', index=False)

```

For best practice we recomend to use **.env** file for DB credential. If you push your notebook to public repo your credential will no expose public.
Please follow best practice for your data security

Go to your folder and create .env file. You can use Vscode or other code editor.

Create below settings.

```
DB_USER=your_username
DB_PASSWORD=your_password
DB_HOST=localhost
DB_PORT=5432
DB_NAME=your_database_name
```
With the below code, system will read your credential to create connection with DB.

```
import os
from dotenv import load_dotenv
import geopandas as gpd
from sqlalchemy import create_engine

# Upload .env  into system
load_dotenv()

# assign .env variable to python variable
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

#  Create SQLAlchemy engine
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

# Read GeoDataFrame
gdf = gpd.read_file('path/to/your/shapefile.shp')

# Pust GeoDataFrame to POSTGIS
gdf.to_postgis(name='your_table_name', con=engine, if_exists='replace', index=False)

print("Success")


```